# 2020 Final Project - MNIST
---
# Board Test for Debugging

In [1]:
from uart import *
from module import *

from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

### Simulation data for board test

In [2]:
# test image number
NUM = 100
input_quan = q8bit_func(np.random.rand(NUM, 28, 28) - 0.5)

### Do inference with software version

In [3]:
out, out_index = sw_inference(input_quan)
# print(out_index)

[array([[[2]]], dtype=int64), array([[[2]]], dtype=int64), array([[[5]]], dtype=int64), array([[[2]]], dtype=int64), array([[[2]]], dtype=int64), array([[[3]]], dtype=int64), array([[[7]]], dtype=int64), array([[[2]]], dtype=int64), array([[[0]]], dtype=int64), array([[[5]]], dtype=int64), array([[[5]]], dtype=int64), array([[[2]]], dtype=int64), array([[[5]]], dtype=int64), array([[[5]]], dtype=int64), array([[[3]]], dtype=int64), array([[[5]]], dtype=int64), array([[[2]]], dtype=int64), array([[[2]]], dtype=int64), array([[[5]]], dtype=int64), array([[[3]]], dtype=int64), array([[[1]]], dtype=int64), array([[[5]]], dtype=int64), array([[[5]]], dtype=int64), array([[[5]]], dtype=int64), array([[[2]]], dtype=int64), array([[[6]]], dtype=int64), array([[[2]]], dtype=int64), array([[[5]]], dtype=int64), array([[[2]]], dtype=int64), array([[[3]]], dtype=int64), array([[[3]]], dtype=int64), array([[[5]]], dtype=int64), array([[[5]]], dtype=int64), array([[[3]]], dtype=int64), array([[[2]]]

In [4]:
print("")
print("board test를 시작합니다.")
print("")
plist = port_list()
for pname in plist:
    try:
        SU = Scale_UART(str(pname))
        print("%s port connected!" %(pname))
        break
    except serial.SerialException:
        print("%s port cannot be connected." %(pname))
print("")
SU.su_flush_buffer()


board test를 시작합니다.

Current OS: Windows
['COM1', 'COM5']
COM1 port cannot be connected.
COM5 port connected!



In [5]:
cycle = 0
correct = 0
wrong_index = []

for i in range(NUM):
    packet = []
    hw_input = convert_to_hw_input(input_quan)
    for j in range(784):
        packet.append(int(hw_input[i*784+j],2))
    SU.snd_packet(packet)
    label, cycle_d = get_info(SU.rcv_packet(5))
    if label == out_index[i]:
        correct += 1
    else:
        wrong_index.append(i)
    cycle += cycle_d
    sys.stdout.write('\r'+str(i+1)+'/'+str(NUM))
    sys.stdout.flush()
print("")
print('Accuracy : ', str(correct/NUM*100), '%')
print('Cycle : ', str(int(cycle/NUM)))
print(f'Wrong Label : {wrong_index}')
print('')
if (correct == NUM):
    print("Board Test에 통과하였습니다! Board_Test_2nd_stage를 진행해 주세요.")
if (correct != NUM):
    print("잘못된 결과가 있습니다. debugging이 필요합니다.")
    print("초기화 문제인지 파악을 위해 RESET(C12) 버튼을 누르고 아래의 코드를 실행해주세요.")

1000/1000
Accuracy :  100.0 %
Cycle :  1274166
Wrong Label : []

Board Test에 통과하였습니다! client test를 진행해 주세요.


### Do inference with FPGA

### Reset FPGA
board test에 실패했다면 아래의 코드를 실행하기 전에 보드를 reset 해주세요.  
xdc 파일을 수정하지 않았다면 CPU RESET (C12)를 누르시면 됩니다.

### Check initialization and Extract debugging data

In [6]:
if len(wrong_index) != 0:
    packet = []
    for j in range(784):
        packet.append(int(hw_input[wrong_index[0]*784+j],2))
    SU.snd_packet(packet)
    label, cycle_d = get_info(SU.rcv_packet(5))
    if label == out_index[i]:
        correct += 1
    else:
        wrong_index.append(i)
    if label == out_index[wrong_index[0]]:
        print("초기화 후 inference 시 정상 동작합니다. 초기화 부분을 확인해주세요.")
    else:
        print("초기화 후에도 값이 틀렸습니다. Debugging용 data를 추출합니다.")
        save_debugging_data(input_quan[wrong_index[0]], out_index[wrong_index[0]])

In [7]:
debug_index = 1
if len(wrong_index) > debug_index:
    print(f'Index :: {wrong_index[debug_index]}')
    packet = []
    for j in range(784):
        packet.append(int(hw_input[wrong_index[debug_index]*784+j],2))
    SU.snd_packet(packet)
    label, cycle_d = get_info(SU.rcv_packet(5))
    if label == out_index[i]:
        correct += 1
    else:
        wrong_index.append(i)
    if label == out_index[wrong_index[debug_index]]:
        print("초기화 후 inference 시 정상 동작합니다. 초기화 부분을 확인해주세요.")
    else:
        print("초기화 후에도 값이 틀렸습니다. Debugging용 data를 추출합니다.")
        save_debugging_data(input_quan[wrong_index[debug_index]], out_index[wrong_index[debug_index]])

Test Using Input_1